In [3]:
from pyspark.sql.functions import coalesce

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("wrangling") \
    .getOrCreate()

In [23]:
realnews_df = spark.read.json("/common/users/ac1771/realnews/realnews.jsonl")

In [16]:
uci_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/news-aggregator-dataset/uci-news-aggregator.csv", header=True)
abc_news_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/a-million-news-headlines/abcnews-date-text.csv", header=True)
all_the_news_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/all-the-news-2/all-the-news-2-1.csv", header=True)
india_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/india-headlines-news-dataset/india-news-headlines.csv", header=True)

In [18]:
print(realnews_df.count() + uci_df.count() + abc_news_df.count() + all_the_news_df.count() + india_df.count())

41974093


In [24]:
realnews_df = realnews_df.select("*", coalesce(realnews_df["summary"], realnews_df["title"]).alias("selected_text"))
realnews_df = realnews_df.drop("split", "status", "text", "url", "url_used", "warc_date", "authors", "domain", "summary", "title")

In [33]:
uci_df = uci_df.select("TIMESTAMP", "TITLE").withColumnRenamed("TIMESTAMP","publish_date").withColumnRenamed("TITLE", "selected_text")
abc_news_df = abc_news_df.withColumnRenamed("headline_text", "selected_text")
india_df = india_df.drop("headline_category").withColumnRenamed("headline_text", "selected_text")
all_the_news_df = all_the_news_df.select("date", "title").withColumnRenamed("date", "publish_date").withColumnRenamed("title", "selected_text")

In [34]:
combined_df = realnews_df.unionByName(uci_df).unionByName(abc_news_df).unionByName(india_df).unionByName(all_the_news_df)

In [36]:
combined_df.count()

41974093

In [37]:
combined_df.write.mode("Overwrite").json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')